In [1]:
from yaulab_processing import run_prepro_and_sorting, clear_tempdir

In [2]:
############################################################################################################################################################################################################
#                                                                       RECORDING / EXPERIMENTAL SESSIONS
############################################################################################################################################################################################################

#########################################################################################################################
#                                        RECORDING PATHS - SETTINGS
#########################################################################################################################
# "parentRecordingFolder"
#  Folder from where to read NWB (read from server, unless you copied the files into the local machine)
parentRecordingFolder = 'Y:\\Data_Albus\\Albus_NeuralData\\Albus-S1'

#########################################################################################################################
#                            SELECT EXPERIMENT DAY AND SESSIONS (listID or None)
#########################################################################################################################

#########################################################################################################################
# Experiment Day : YEAR, MONTH, DAY
sessionYear = 2024
sessionMonth = 4 
sessionDay = 25

#########################################################################################################################
# ElectrodeGroupe Name (based on NWBfile naming of the "acquisition" container) :
# example: 'raw-C1-257-288'
# Note: if there were different coordinates within the same day, you need to add '-sub1', '-sub2', etc.
# It is recommended to run "run_prepro" first, it will create folder for all the unique electrode groups from that day.
# If run_prepro has benn run, then check the "electrodeGroupe_Name" by exploring the "parentPreproSortingFolder"
# In case you want to explore how many electrodeGroup-subX exists, run:
#       from yaulab_processing.yaulab_si.spikeinterface_tools import print_recordingInfo
#       print_recordingInfo(parentRecordingFolder, sessionYear, sessionMonth, sessionDay)
###########################################################################################################################
electrodeGroup_Name = 'raw-C1-257-288' # 'raw-C1-257-288' 'raw-D1-385-416'

#########################################################################################################################
# Session index is based on the dateTime. Zero is the first session of the day from the selected electrodeGroup
# If None, it will use all sessions from this electrodeGroup in this experiment Day
# If you want specify sessions it has to be a list of indexes, it can be non-consecutive [0, 1, 2] 
session_index = None



In [3]:
############################################################################################################################################################################################################
#                                                                           PREPROCESSING
############################################################################################################################################################################################################

#########################################################################################################################
# "parentPreproSortingFolder":
#  Folder to save all the Plots & recording objects from the preprocessing (server is recommended to keep record of the sorting process)
parentPreproSortingFolder = 'Y:\\Data_Albus\\Albus_NeuralData\\Albus-S1_preproSorting' 

#########################################################################################################################
# "rewrite_prepro" : 
#       if TRUE it will run all the preprocessing regardless of finding the previous recording objects
#               if you are going to change some preprocessing params, set it to True
#       if FALSE it will check whether preprocessed and motion_iterpolated recordings exist 
#               if those exists, it will be reloaded and pass it to the sorter
#               if those not exists then it will run the preprocessing and pass it to the sorter
rewrite_prepro = False

#########################################################################################################################
#                                         GENERAL PARAMETERS:
#########################################################################################################################

#########################################################################################################################
# PARAMETER FOR "LOCAL REFERENCING"
exclude_radius_chans = 1 # Number of neighbor channels to exclude because are too close to the reference channel
include_radius_chans = 4 # Number of neighbor channels delineates the outer boundary of the annulus whose role is to exclude channels that are too far away

#########################################################################################################################
# NOISY FREQUENCY TO BE REMOVED (i.e., "spre.notch_filter")
noisy_freq = None
# TODO: bandpass filter

#########################################################################################################################
# DETECT PEAKS: 
peak_detect_threshold = 5 #  MAD: Median Amplitude Deviations : Default: 5
peak_sign = 'both' # (“neg” | “pos” | “both”) Default: 'both'
nearest_chans = 1 # Number of neighbor channels to search for events that occurred at the same time. Default: 2

#########################################################################################################################
# GENERAL PARAMETER FOR "PEAKS LOCATIONS" It will also be use for sorting 
ms_before = 0.6 # Default: 0.6
ms_after = 1.5 # Default: 1.5

######################################################################################################
# Detect and  Remove bad channels ('coeherence+psd' : good/dead/noise/out )
# It will remove dead, noise, out channels. 
# NOTE: Sometimes channels with too much high-amplitude activity will be classify as dead or noisy. 
# Check the recording traces before setting this to False (it will label all channels as good)
do_detect_bad_channels = True # (True | False) Default: True

#########################################################################################################################
#                   MOTION OPTIONS to run : non-rigid + decentralized + kriging
#########################################################################################################################

#########################################################################################################################
# MOTION ESTIMATION
# if TRUE it will do motion estimation and interpolation.
do_motion = True # Default: True

#########################################################################################################################
# Rigid or nonRigid
motion_rigid = False # (True | False) Default: False

#########################################################################################################################
# Motion correction: Paninski Lab
motion_options = {
    'method': 'dredge_ap', # Paninski Lab
    'method_kwargs' : {} 
}

#########################################################################################################################
# Interpolation: 'kriging' - Kilosort-like
interpolate_options = {
    'method': 'kriging', # ('kriging' | 'idw' | 'nearest') 'idw': inverse distance weighted.  Default: 'kriging' - Kilosort-like
    'border_mode': 'remove_channels' # ('remove_channels' | 'force_extrapolate' | 'force_zeros') Default: 'remove_channels'
}

In [6]:
############################################################################################################################################################################################################
#                                                                                SORTING
############################################################################################################################################################################################################

#########################################################################################################################
#                                               SORTING PATHS - SETTINGS
#########################################################################################################################
# "parentSortingFolder" : folder to save PHY & "recording" (binary) & "sorting_analyzer"
#  if "parentSortingFolder" is not provided, then "parentPreproSortingFolder" will be used as a default.
parentSortingFolder =  None 

# If False, it will only run preprocessing 
run_sorting = True

#########################################################################################################################
#                                               SORTING PARAMETERS
#########################################################################################################################

######################################################################################################
# SORTER NAME (options: waveclus, mountainsort5 & kilosort4)
# Sorter selection based on channel counting:
# "waveclus" or "mountainsort5": less than equal 4 channels in the electrodeGroup
# "kilosort4" or "mountainsort5": above 4 channels in the electrodeGroup
sorterName = 'kilosort4'

######################################################################################################
# SORTER "Sign" to detect Events
# Note: , set this to 'both' for kilosort4 (KS4 uses absolute detection threshold) specially if you postprocessing_alingSorting = True 
# This value will be use to compute all the SortingAnalyzer metrics 
sorter_detect_sign = 'both'  # (“neg” | “pos” | “both”) Default: 'both'

######################################################################################################
# SORTER THRESHOLD
# waveclus: STD units from whithening traces (default = 5) [you can try values as low as 1.5] Default "Sign" to detect Events is the same as the one used in DETECT PEAKS
# mountainsort5: ZCA-Whitening units (default = 5.5) [you can try with values as low as 3] Default "Sign" to detect Events is the same as the one used in DETECT PEAKS
# kilosort4: STD units from absolute whithened traces (default = 3) [you can try values as low as 1.5]
# If None, it will use default values according to each sorting
sorter_detect_threshold = 3

######################################################################################################
# Number of NEIGHBOR CHANNELS to search for the same waveform during sorting & sorting_analyzer
# "nearest_chans = 0" will perform similar to sorting single channels. KS$ do not accept 0, minumum 1
# If set to None, it will use the "nearest_chans" value from peaks locations
# Note: Nearest channel is not use by waveclus sorting, but it will be used to create the sorting_analyzer
sorter_nearest_chans = 1 # Default 1

######################################################################################################
# ALIGN SORTER: In some situations spike sorters could return a spike index with a small shift related to the waveform peak
# See : https://spikeinterface.readthedocs.io/en/latest/api.html#spikeinterface.core.get_template_extremum_channel_peak_shift
# Recommended to set to "True" for Mountainsort5
postprocessing_alingSorting = False


In [ ]:
folder_temporal = run_prepro_and_sorting(
            parentRecordingFolder = parentRecordingFolder, 
            parentPreproSortingFolder = parentPreproSortingFolder, 
            sessionYear = sessionYear, 
            sessionMonth = sessionMonth, 
            sessionDay = sessionDay,  
            electrodeGroup_Name = electrodeGroup_Name, 
            session_index = session_index,
            parentSortingFolder = parentSortingFolder,
            local_radius_chans = (exclude_radius_chans, include_radius_chans), 
            noisy_freq = noisy_freq, 
            ms_before = ms_before, 
            ms_after = ms_after, 
            peak_sign = peak_sign,
            nearest_chans = nearest_chans, 
            peak_detect_threshold = peak_detect_threshold, 
            do_detect_bad_channels = do_detect_bad_channels,
            do_motion = do_motion,
            motion_rigid = motion_rigid, 
            motion_options = motion_options, 
            interpolate_options = interpolate_options,
            rewrite_prepro = rewrite_prepro,
            run_sorting = run_sorting,
            sorterName = sorterName, 
            sorter_detect_sign = sorter_detect_sign,
            sorter_detect_threshold = sorter_detect_threshold,
            sorter_nearest_chans = sorter_nearest_chans,
            postprocessing_alingSorting = postprocessing_alingSorting,
        )


In [6]:
##################################################################################################################################################
# To ensure all temporary files are deleted this section must be run after the function is done
# If you want to run another sorting Or change some parameter, do it before running this cell and deleting the temporal folder. 
# It will avouid writing the WHITENED recording into binary twice.

# NOTE: If there is a message that some files are still open, restart the notebook and DELETE manually the temporal folder

print('\nDeleting temporal Folder .............\n')
clear_tempdir(tempFolderPath=folder_temporal, verbose=True)